# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fÃ©dÃ©rale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.



In [78]:
# Import libraries
import requests
from bs4 import BeautifulSoup

from selenium import webdriver # to have the javascript processed version of the HTML

import json
import pandas as pd

In [44]:
# Make the request
URL = 'https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!'\
                 +'/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats'
    
URL2 = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
    

In [45]:
r = requests.get(URL)
r2 = requests.get(URL2)
print('r = {r} // status_code = {status}'.format(r=r,status=r.status_code))
print('r2 = {r} // status_code = {status}'.format(r=r2,status=r2.status_code))

r = <Response [200]> // status_code = 200
r2 = <Response [200]> // status_code = 200


In [79]:
r2.content
soupp = BeautifulSoup(r2.content,'html.parser')
print(soupp.prettify())

{"data":[{"rank_order":"10","rank":"1","name":"University of Oxford","scores_overall":"94.3","scores_overall_rank":"10","scores_teaching":"86.7","scores_teaching_rank":"5","scores_research":"99.5","scores_research_rank":"1","scores_citations":"99.1","scores_citations_rank":"15","scores_industry_income":"63.7","scores_industry_income_rank":"169","scores_international_outlook":"95.0","scores_international_outlook_rank":"24","record_type":"master_account","member_level":"0","url":"\/world-university-rankings\/university-oxford","nid":468,"location":"United Kingdom","stats_number_students":"20,409","stats_student_staff_ratio":"11.2","stats_pc_intl_students":"38%","stats_female_male_ratio":"46 : 54","aliases":"University of Oxford","subjects_offered":"Archaeology,Art, Performing Arts \u0026 Design,Biological Sciences,Business \u0026 Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics \u0026 Econometrics,Electrical \u0026 Electronic Engineering,General Engi

In [149]:
#lm_json = requests.get(URL2).json() ## not working for the moment
#soupy = BeautifulSoup(lm_json['html'])

KeyError: 'html'

In [145]:
rank_json = json.loads(r2.text)
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
#rank_df.stars
#rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
#rank_df.set_index('core_id', inplace=True)
rank_df.head()

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [46]:
driver = webdriver.PhantomJS()
driver.get(URL)

# This will get the initial html - before javascript
html1 = driver.page_source

# This will get the html after on-load javascript
html2 = driver.execute_script("return document.documentElement.innerHTML;")

In [132]:
soup = BeautifulSoup(html2,'html.parser')
print(soup.prettify())

<head class="jquery-once-1-processed" profile="http://www.w3.org/1999/xhtml/vocab">
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="app-id=578411515" name="apple-itunes-app"/>
 <link href="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
 <meta content="summary_large_image" name="twitter:card"/>
 <meta content="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/img/social/ranking-dataset-share.jpg" name="twitter:image"/>
 <meta content="World University Rankings 2018" name="twitter:title"/>
 <meta content="@timeshighered" name="twitter:site"/>
 <meta content="The Times Higher Education World University Rankings 2018 list the top 1,000 universities in the world, making it our biggest international league table to date. It is the

In [133]:
list(soup.children)

[<head class="jquery-once-1-processed" profile="http://www.w3.org/1999/xhtml/vocab">
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="app-id=578411515" name="apple-itunes-app"/>
 <link href="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
 <meta content="summary_large_image" name="twitter:card"/>
 <meta content="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/img/social/ranking-dataset-share.jpg" name="twitter:image"/>
 <meta content="World University Rankings 2018" name="twitter:title"/>
 <meta content="@timeshighered" name="twitter:site"/>
 <meta content="The Times Higher Education World University Rankings 2018 list the top 1,000 universities in the world, making it our biggest international league table to date. It is th

In [134]:
r.text
soup.title
soup.title.string

'World University Rankings 2018 | Times Higher Education (THE)'

In [137]:
all_links = soup.find_all('a')
print('The webpage cointains {0} links...'.format(len(all_links)))
for link in all_links:
    print(link.get('href'))

The webpage cointains 2316 links...
#main-content
/
/
/
/academic/news
/academic/opinion
/comment/reader-comments
/academic/books
/academic/features
http://www.the-awards.co.uk
/academic/digital-editions
https://www.timeshighereducation.com/unijobs
https://www.timeshighereducation.com/unijobs/listings/
https://www.timeshighereducation.com/unijobs/newalert/
https://www.timeshighereducation.com/unijobs/employers/
/policy/careers
/summits
/world-university-rankings
/world-university-rankings/2017/world-ranking
/rankings/united-states/2017
/rankings/japan-university/2017
/world-university-rankings/by-subject
/policy/rankings
/world-university-rankings/about-the-times-higher-education-world-university-rankings
/student
/student/best-universities
/student/where-to-study
/student/news
/student/advice
/student/blogs
/search/type/ranking_institution
/student/subjects
/about-us
#registerModal
/the_user/password
/cookie-policy
/terms-and-conditions
#loginModal


/store
None
https://www.timeshighe

In [138]:
all_links = soup.find_all('link')
print('The webpage cointains {0} links...'.format(len(all_links)))

The webpage cointains 11 links...


In [139]:
all_links = soup.find_all('script')
print('The webpage cointains {0} links...'.format(len(all_links)))

The webpage cointains 13 links...


In [140]:
ll=soup.find_all('td')
print(len(ll))
ll[1]

2204


<td class=" name namesearch"><a class="ranking-institution-title" data-mz="" data-position="title" href="/world-university-rankings/university-oxford#ranking-dataset/629337">University of Oxford</a><div class="location"><span><i class="fa fa-map-marker"></i>United Kingdom</span></div><a class="btn btn-gray btn-margin" data-mz="" data-position="explore" href="/world-university-rankings/university-oxford#ranking-dataset/629337">Explore</a></td>

In [141]:
right_table = soup.find('table',id="datatable-1")
rows = right_table.find_all('tr',role='row')#role="row")
print(len(rows))
rows[1].findAll('td')

1103


[<td class="rank sorting_1 sorting_2">1</td>,
 <td class=" name namesearch"><a class="ranking-institution-title" data-mz="" data-position="title" href="/world-university-rankings/university-oxford#ranking-dataset/629337">University of Oxford</a><div class="location"><span><i class="fa fa-map-marker"></i>United Kingdom</span></div><a class="btn btn-gray btn-margin" data-mz="" data-position="explore" href="/world-university-rankings/university-oxford#ranking-dataset/629337">Explore</a></td>]

In [143]:
A=[]
B1=[]
B2=[]
C=[]
D=[]
E=[]
F=[]
G=[]
for r in rows[1:10]:
    cells = r.findAll('td')
    if len(cells)>0:
        A.append(cells[0].find(text=True))
        univ = cells[1].find('a').find(text=True)
        loc =  cells[1].find('div').find(text=True)
        B1.append(univ)
        B2.append(loc)
        #C.append(cells[2].find(text=True))
        #D.append(cells[3].find(text=True))
        #E.append(cells[4].find(text=True))
    #if len(cells)==6: #Only extract table body not heading
    #A.append(cells[0].find(text=True))
print(A)
print(B1)
print(B2)
print(C)
print(D)
print(E)


['1', '2', '=3', '=3', '5', '6', '7', '8', '9']
['University of Oxford', 'University of Cambridge', 'California Institute of Technology', 'Stanford University', 'Massachusetts Institute of Technology', 'Harvard University', 'Princeton University', 'Imperial College London', 'University of Chicago']
['United Kingdom', 'United Kingdom', 'United States', 'United States', 'United States', 'United States', 'United States', 'United Kingdom', 'United States']
[]
[]
[]


In [144]:
df=pd.DataFrame(A,columns=['Rank'])
df['University']=B1
df['Location']=B2
df

,Rank,University,Location
0,1,University of Oxford,United Kingdom
1,2,University of Cambridge,United Kingdom
2,=3,California Institute of Technology,United States
3,=3,Stanford University,United States
4,5,Massachusetts Institute of Technology,United States
5,6,Harvard University,United States
6,7,Princeton University,United States
7,8,Imperial College London,United Kingdom
8,9,University of Chicago,United States


In [131]:
for r in rows[:10]:
    print(r.find('td', class_=" stats stats_number_students"))

None
None
None
None
None
None
None
None
None
None
